#GST Colloquium on Python coding


*   random grab-bag of tips and tricks
*   hyper-optimization is not encouraged in Python but these techniques can provide large performace gains for little-to-no extra work
*   In many cases these optimizations don't matter much and we've chosen small examples for practicality but when working on large datasets they can provide major practical improvements



In [ ]:
#TESTING ONLY
#this allows us to interact with google drive from within python
from google.colab import drive
#we mount our google drive which means that we make it acessable as a hard drive
#in our file system under the path '/content/drive'
drive.mount('/content/drive')
!cp /content/drive/MyDrive/GST_colloquium/* .

MessageError: Error: credential propagation was unsuccessful

In [ ]:
#colab comes with most common packages installed already but we want to use a
#few uncommon ones so we install these ourselves.
#The "!" tells jupyter to pass the line to the bash shell, this allows us to run
#command line tools like pip from within jupyter notebook.
!pip install brain-isotopic-distribution
!pip install multiprocesspandas
!pip install xlsx2csv

In [ ]:
#it is generally recommended best practice to import the packages that come with
#python first before importing third party packages.
from collections import defaultdict
from functools import cache
from multiprocessing import Pool
import time
import os
import warnings
import re
from io import StringIO

from brainpy import isotopic_variants
from numba import jit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocesspandas
from scipy.optimize import minimize
from scipy.stats import zscore
from scipy.stats import binom
from sortedcontainers import SortedList
from xlsx2csv import Xlsx2csv
from sklearn.metrics import mean_squared_error

In [ ]:
#This defines a function decorator, a concept which is used in §Multiprocessing: Numba
#It times a function call and prints the elapsed time while suppressing warnings
def time_this(func):
  #the way this works is that time_this() creates and returns a new function
  #that replaces whatever funciton follows @time_this. But, because
  def inner(*args, **kwargs):
    #this context manager suppresses all warnings. DO NOT DO THIS NORMALLY
    #We have extensively tested this code and data and know that the warnings
    #can be safely ignored for this tutorial but that is not true for most code
    with warnings.filterwarnings('ignore'):
      start_time = time.process_time()
      result = func(*args, **kwargs)
      elapsed_time = time.process_time() - start_time
    print(f'Running {func.__name__} took {elapsed_time} seconds.')
    return result
  return inner

#A test version without the warning suppression
def time_this(func):
  def inner(*args, **kwargs):
    start_time = time.process_time()
    result = func(*args, **kwargs)
    elapsed_time = time.process_time() - start_time
    print(f'Running {func.__name__} took {elapsed_time} seconds.')
    return result
  return inner

##Data Structures

The way data are stored in memory can have an enormous effect on the speed with which they can be processed for particular tasks. Choosing the correct data structure can be the difference between a runtime of hours or minutes in some situations.

###Hashmaps

Hashmaps are data structures that store key value pairs in a way that allows you to very quickly look up a value based on a key. Dictionaries in python are hashmaps which can store any object as a value and can use any hashable object as a key. Sets are specialized hashmaps which only store true or false as values. These can be used to efficiently look up whether or not a key is in the set.

A hash is a function that maps objects of arbitrary size and complexity to a finite range of numbers. In the context of a hashmap data structure the finite range represents locations in memory and the hash function calculates where in memory a value is stored based on its key which means that the time it takes to find a value is only dependent on the time it takes to calculate the hash of a key. This contrasts with looking up values in a list which requires iterating over each item in the list until you find what you're looking for, an operation that becomes more expensive on average as the list grows in size.


<img align="left" src="https://github.com/m-j-keller/GST_colloquium/blob/main/Hashmapfigure-margin.png?raw=true" alt="Figure 1. Hashmap Data Call" title="Figure 1: Accessing Data in a Hashmap" width="440" hspace="20" vspace="10"> The tradeoff that you have to make with hashmaps is that there are some cases in which two keys have the same hash so their values must be stored in a list and iterated over for lookup. To minimize this problem the range of memory locations needs to be significantly larger than the number of values stored. This means that hash maps will generally be larger in memory than more compact data structures like lists. This is not as much of a problem as it first appears because  Python does not store the actual value at the memory location calculated by the hash function. Instead the memory location it calculates is the index of a list that stores memory addresses which point to where the values are actually stored. This means that each block of memory indexed by the hashmap can be very small while still being able to point to values of arbitrary size. Therefore the memory overhead is only likely to be siginficant if you're storing a large number of small things.





**TLDR:** Storing data in hashmaps (like a python dictionary) can significantly improve data access speed, thereby speeding up the whole script.





###Search trees

###Example 1: fuzzy matching

###Example 2: file parsing

###Example 3: isotope abundance calculations

##Pandas and Numpy

###Faster .xlsx import
Large .xlsx files can be slow to import with pandas. This is because they are actually zipped folders of mostly XML files which need to be decompressed before going through the somewhat inefficient process of XML parsing. This setup gives excel the flexability to store images, graphs, formatting, and functions but, if all you want is a table of data, this flexability comes at significant performance costs. Thus it is generally preferable to store tabular data as a text file (.csv or .tsv) unless you absolutely need to store these   extra types of information.

If you have no choice but to use large .xlsx files there is still a faster way to import the data as a pandas dataframe. The xlsx2csv package is sigificantly faster at parsing .xlsx files than pandas. So, what we can do is parse the .xlsx file to an in-memory .csv (meaning that the file is stored only in RAM and not written to our hard drive) then use the very efficient pandas .csv parser to get the data into a dataframe. This trick comes from [stackoverflow](https://stackoverflow.com/questions/28766133/faster-way-to-read-excel-files-to-pandas-dataframe#comment136215400_28769537).

In [ ]:
@time_this
def fast_import(file_path):
    buffer = StringIO()
    Xlsx2csv(file_path, outputencoding="utf-8").convert(buffer,sheetid=0)
    buffer.seek(0)
    df = pd.read_csv(buffer, low_memory=False, skiprows=1)
    return df

@time_this
def pandas_import(file_path):
  df = pd.read_excel(file_path)
  return df

In [ ]:
fast_df = fast_import('metabolomics_data.xlsx')
pandas_df = pandas_import('metabolomics_data.xlsx')

Running fast_import took 347.360885367 seconds.
Running pandas_import took 872.374947258 seconds.


###Example 4: metabolomics data

###Using vectorized operations
Numpy carries out most data processing operations using highly optimized pre-compiled code that is orders of magnitude faster than anything Python can natively accomplish. The optimizations include single instruction, multiple data (SIMD) steps in which a single operation that would normally be applied serially to values in an array can be applied simultaniously to blocks of 4-8 values. When doing mathematical operations it is therefore best to let numpy handle as much of the processing as possible. This means that we should avoid writing explicit loops or accessing individual values within numpy arrays as much as possible and instead perform operations on whole arrays at once using numpy functions.

This strategy holds true for other packages like Pandas, Scipy, and Scikit-Learn as well because Numpy provides most of the data processing functionality for these packages.  

###Example 5: normalization
Proteomic normalization and imputation are important to minimize non-biological variation and ensure reliability of your dataset. Normalization is used to minimize variation coming from factors such as sample preparation inconsistencies, injection volume errors, and fluctuations in instrument sensitivity, which can happen with extended use (i.e. sensitivity drift with instrument cleanliness, or column age). By accounting for these variations, normalization allows for the comparison of biological variances rather than technical ones. Following normalization, imputation fills missing values with low abundance numbers that mimic the instrument's noise floor, allowing for statistical testing across conditions. Both normalization and imputation create a baseline from which conditions can be compared in a more robust way.


In [ ]:
#The data processing produces one file per condition that we read into a list
#of dataframes
prot_files = [f for f in os.listdir() if f.endswith('Proteins.txt')]
dataframes = [pd.read_csv(f, sep = '\t') for f in prot_files]
key_cols = ['Accession','Description','MW [kDa]','# AAs']
#We need to merge these data into a single dataframe for further processing
proteins = dataframes.pop()
for df in dataframes:
  #on = key_cols is the list of columns that we use as a key for which rows to merge
  #how = 'outer' means we use the union of these keys
  proteins = proteins.merge(df, how = 'outer', on = key_cols)
#we want a list of the columns containing protein abundances for processing
abundance_cols = [c for c in proteins.columns if c.startswith('Abundance')]
#For this example we only want to work with the key and abundance columns
#so we get rid of all the other data in this dataframe.
#The * operator unpacks an interable so [1,*[2,3,4]] becomes [1,2,3,4].
proteins = proteins[[*key_cols, *abundance_cols]]

In [ ]:
rng = np.random.default_rng(1)

@time_this
def vector_norm_impute(values):
  #processing is done in the log space because intensity is log-normally
  #distributed across proteins
  values = np.log(values)
  #we calculated the mean and standard deviation of all values in all samples
  gmean = np.nanmean(values)
  gstd = np.nanstd(values)
  #intensities are z-scored by column (axis = 0), which corresponds to files
  values = zscore(values, axis = 0, nan_policy = 'omit')
  #we un-do the z-scoring with the grand mean and standard deviation so that
  #all files now have the same log-mean and log-standard deviation
  values = (values*gstd)+gmean
  #we impute by taking draws from a normal distribution centered 1.8 standard
  #deviations below the mean to simulate the correlation between missingness and
  #intensity
  loc = gmean - (1.8*gstd)
  scale = gstd*0.3
  #The strategy here is to generate a random number for each value in our data
  #and then only transfer random values to our data at the missing value
  #positions. We do generate significantly more random values than necessary
  #this way but the vectorized approach is so much faster than repeated calls to
  #rng that it is still the preferred approach
  imputed_values = rng.normal(loc, scale, values.shape)
  mask = np.isnan(values) #an array of true/false for if a value is missing
  values[mask] = imputed_values[mask] #only missing values are replaced
  #the data are taken back out of log space for downstream processing
  values = np.exp(values)
  return values

@time_this
def loop_norm_impute(values):
  #this is how you do a nested loop within a list comprehension
  logvalues = [np.log(value) for sample in values for value in sample]
  #as above we pre-calculate variables that depend on all values
  gmean = np.nanmean(logvalues)
  gstd = np.nanstd(logvalues)
  loc = gmean - (1.8*gstd)
  scale = gstd*0.3
  #in this verison we construct a list of lists
  new_values = []
  for i in range(values.shape[1]):
    sample = values[:,i]
    new_sample = []
    #in order to do the normalization elementwise we have to precalculate
    #sample-specific values
    smean = np.nanmean(sample)
    sstd = np.nanstd(sample)
    for value in sample:
      #all the processing from here out is the same but done on one element
      #at a time
      if np.isnan(value):
        new_sample.append(rng.normal(loc, scale))
      else:
        value = np.log(value)
        zscore = (value-smean)/sstd
        value = (zscore*gstd)+gmean
        value = np.exp(value)
        new_sample.append(value)
    new_values.append(new_sample)
  #the .T is a transpose function which swaps rows for columns
  return np.array(new_values).T


In [ ]:
vec_proteins = proteins.copy()
vec_proteins[abundance_cols] = vector_norm_impute(vec_proteins[abundance_cols].to_numpy())
loop_proteins = proteins.copy()
loop_proteins[abundance_cols] = loop_norm_impute(loop_proteins[abundance_cols].to_numpy())

Running vector_norm_impute took 0.013887138999990611 seconds.
Running loop_norm_impute took 0.45028108699997915 seconds.


###Building and accessing dataframes

###Example 6: protein summary statistics

In [ ]:
@time_this
def numpy_stats(df):
  df['mean'] = np.nanmean(df[abundance_cols], axis = 1)
  df['std'] = np.nanstd(df[abundance_cols], axis = 1)
  return df

@time_this
def listcomp_stats(df):
  abundance_iter = list(zip(*[df[c] for c in abundance_cols]))
  df['mean'] = [np.nanmean(row) for row in abundance_iter]
  df['std'] = [np.nanstd(row) for row in abundance_iter]
  return df

@time_this
def iterrows_stats(df):
  means = []
  stds = []
  for i,row in df[abundance_cols].iterrows():
    means.append(np.nanmean(row))
    stds.append(np.nanstd(row))
  df['mean'] = means
  df['std'] = stds
  return df

@time_this
def at_stats(df):
  df['mean'] = ''
  df['std'] = ''
  for i,row in df[abundance_cols].iterrows():
    df.at[i,'mean'] = np.nanmean(row)
    df.at[i,'std'] = np.nanstd(row)
  return df


In [ ]:
numpy_df = numpy_stats(proteins.copy())
listcomp_df = listcomp_stats(proteins.copy())
iterrows_df = iterrows_stats(proteins.copy())
at_df = at_stats(proteins.copy())

In [ ]:
@time_this
def melt(df):
  df = df.melt(id_vars = key_cols, value_vars = abundance_cols)
  return df

@time_this
def loop_melt(df):
  dfs = [ ]
  for a in abundance_cols:
    tmp_df = df[[*key_cols,a]].copy()
    tmp_df.columns = [*key_cols,'Abundnace']
    tmp_df['sample'] = [a]*df.shape[0]
    dfs.append(tmp_df)
  df = pd.concat(dfs)
  return df

@time_this
def accumulator_melt(df):
  new_df = pd.DataFrame()
  for a in abundance_cols:
    tmp_df = df[[*key_cols,a]].copy()
    tmp_df.columns = [*key_cols,'Abundnace']
    tmp_df['sample'] = [a]*df.shape[0]
    new_df = pd.concat([new_df, tmp_df])
  return new_df

In [ ]:
melt_df = melt(proteins.copy())
print(melt_df.shape)
loop_melt_df = loop_melt(proteins.copy())
print(loop_melt_df.shape)
accum_melt_df = accumulator_melt(proteins.copy())
print(accum_melt_df.shape)


Running melt took 0.02813604199999986 seconds.
(108306, 6)
Running loop_melt took 0.04736300299998675 seconds.
(108306, 6)
Running accumulator_melt took 0.06905706599999917 seconds.
(108306, 6)


##Black Box Optimizers

###scipy.optimize

Optimizers are useful tools for a variety of tasks. If you ever find yourself doing a "guess and check" procedure, consider automating it with an optimizer function. The scipy package contains a variety of off-the-shelf black box optimizers that are easily applicable to a variety of situations. All you need is a way of numerically summarizing task "success". This is accomplished with a loss function (function to calculate error). The optimize will start with an initial guess you provide and iteratively adjust the value searching for an error minimum.

**Possible Pitfall:** If your initial guess is too far off, the scipy optimizer can give pathological results, without nessesarily throwing an error.



###Example 7: isotopic packet fitting
In one Hettich-lab project, it was nessesary to estimate the level of lipid deuteration by fitting a binomial distribution to isotope abundance data. Given the setup of the experiment, the binomial probability would correspond to the deuteration percent in the lipid. Someone whose name is definitely **not** Matthew built an excel spreadsheet to generate binomial distributions from a given probability. Matthew--\*ahem\*--this nameless person proceeded to generate binomial distributions from a variety of probabilities, plot them against the observed data, and generate new probabilites based on the perceived goodness of fit.

This can be done more precisely and much faster using scipy.optimize.minimize. Basically the data are fed into minimize() which then changes the binomial probablility a bunch of times until it identifies a minimum point. In this case, we are calculating the root-mean squared error in our loss function.

In [ ]:
class hashabledict(dict): #Needed to prevent collisions when memoizing formulae
    def __hash__(self):
        return hash(tuple(sorted(list(self.items()), key = lambda x: x[0])))

@cache
def formparser(chemformula): #Custom chemical formula reader. Only works for single letter isotopes; This version uses memoize to speed up if there are many calculations with repeated formulae
    chemform = chemformula.strip(" ")
    parsed = re.findall(r'\D\d*',chemform)
    composition = {p[0]:int(p[1:]) if len(p) > 1 else 1 for p in parsed}
    #return(hashabledict(composition))
    return(composition)

def deu_binom(totalhydrogen, pD, nonexH = 3): #Generates a binomial distribution; need to adjust nonexH based on how many hydrogen in the formula are expected to back exchange with the solvent and be unlabeled
    lit = []
    for n in range(totalhydrogen-nonexH+1):
        lit.append(binom.pmf(n, totalhydrogen-nonexH, pD))
    arr = np.array(lit)
    return arr

@cache
def natisodist(formuladict): # Generates natural isotope distribution to correct naive binomial model; uses memoize to potentially speed processing
    theoretical_isotopic_cluster = isotopic_variants(formuladict, npeaks=5, charge=1)
    array = np.empty(0)
    for peak in theoretical_isotopic_cluster:
        array = np.append(array, peak.intensity)
    return(array)

def natcor_deudist(formula, pD, scaled = True): #Combines the calculated natural isotope distribution with the deuterium distibution
    pD = float(pD)
    formdict = hashabledict(formparser(str(formula)))
    nH = formdict.pop('H')
    natdist = natisodist(formdict)
    deudist = deu_binom(nH, pD)
    totdist = np.convolve(natdist, deudist)
    if scaled:
        totdist = totdist/max(list(totdist))
    return(totdist)

def error(pD,formula,actualdata): #Loss function for the deuterium fitting. Calculates the root mean squared error between the generated distribution and the actual data.
    '''
    calculate expected dist from elements given using above function.
    calculate RMS error from actualdata as compared to calculated data.
    '''
    expecteddata = natcor_deudist(formula,pD)
    actualdataext = np.append(actualdata, [0,0,0,0,0,0])
    actualdatatrim = actualdataext[:len(expecteddata)]
    return(mean_squared_error(actualdatatrim, expecteddata, squared=False))

def optimize(pD, formula, actualdata): #Optimization function that fits a natural isotope-corrected binomial distribution to observed data
                                       #Here 'Nelder-Mead' is the specific optimization algorithm used. Sometimes specific data will break a given method, if you have issues, try changing the method.
    '''
    Use err function for least squared optimization of pD.
    Takes RMS error calculated by err and varies pD to minimize RMSE
    Returns least_squares optimization output
    Needs pD to be in the ballpark of the best value, otherwise behaviour can be weird
    Assumes Carbon, Hydrogen, Oxygen, Nitrogen, and actualdata are defined prior to using optimize
    '''
    results = minimize(error, x0 = pD, bounds = [[0,1]], method = 'Nelder-Mead', args=(formula, actualdata))
    param = results.x
    return(results)

@time_this
def deuterium_fitting(df): #Wrapper function to fit deuterium distributions to a table of lipids and append the results
    int_cols = [*df.columns[7:]]
    int_iter = list(zip(*[df[c] for c in int_cols]))
    pDs = list(df["pD"])
    formula = list(df["Formula"])
    test = [optimize(pD, form, row) for row, pD, form in zip(int_iter, pDs, formula)] #This could be done with iterrows, but as shown earlier, the list comprehension is slightly faster
    maxiso = int(str(df.iloc[:,-1].name)[1:])
    df["BestFit pD"] = [t['x'][0] for t in test]
    df["RMSE"] = [t['fun'] for t in test]
    df["fun termination"] = [t['message'] for t in test]
    def exp_dist(formula, x):
        dist = natcor_deudist(formula, x)
        dist = np.concatenate((dist,np.zeros(maxiso - len(dist))))
        return dist
    dists = np.array([exp_dist(f, t['x'][0]) for f,t in zip(formula, test)])
    dists = pd.DataFrame(dists, columns= [f'isotope_{i}' for i in range(dists.shape[1])])
    df = pd.concat([df,dists], axis = 1)
    return df

df = pd.read_csv("DL_Example_3-25-24.csv", sep=',', encoding="UTF-8")

df = deuterium_fitting(df)


ModuleNotFoundError: No module named 'brainpy'

**TLDR:** Pre-built algorithms, like scipy's minimize function, offer a powerful solution to otherwise slow manual "guess and check" operations.

<img align="left" src="https://github.com/m-j-keller/GST_colloquium/blob/main/DataFitting.png?raw=true" alt="Figure 3. Fitting Binomial to Isotope Distribution" title="Figure 1: Accessing Data in a Hashmap" width="400" hspace="20" vspace="20">


##Multiprocessing

###Numba
The standard implementation of python is interpreted. This means that when python code is executed a program called the interpreter goes through the code line-by-line and, by way of an intermediate representation, runs just that line. This has the advantage of flexability but it requires sigificant overhead and prevents the interpreter from analyzing and optimizing your code. This is in contrast with compiled languages where all of the code is read, analyzed, optimized, and turned into machine code that your CPU can understand before anything is run.

Numba gives us a stepping stone between these two strategies. It is a just-in-time compiler for python funcitons. This allows us to take performance intensive funcitons and compile them into optimized machine code at runtime. For performance reasons not all python features are available within numba but if your function can be written with a combination of base python and numpy then it can proabably be optimized with numba.

Another advantage of numba is that the compiled code is not subject to the GIL so calculations can be written to run in parallel.  [Steven should define GIL]

###Example 8: custom correlation matrix

###multiprocessing.Pool

###Example 9: bootstrapping
